In [4]:
import pandas as pd

# Load your training data
df = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/train.csv")

# Create the text_input column by combining prompt + response_a + response_b
df["text_input"] = (
    "Prompt: " + df["prompt"] + 
    " \nResponse A: " + df["response_a"] + 
    " \nResponse B: " + df["response_b"]
)

# Create label based on winner_a and winner_b
def decide_winner(row):
    if row["winner_model_a"] > row["winner_model_b"]:
        return "A"
    elif row["winner_model_b"] > row["winner_model_a"]:
        return "B"
    else:
        return "winner_tie"

df["label"] = df.apply(decide_winner, axis=1)

# Keep only the necessary columns
df = df[["text_input", "label"]]

# Save the formatted file
df.to_csv("train_formatted.csv", index=False)

print("Saved formatted training data to train_formatted.csv")

Saved formatted training data to train_formatted.csv


In [6]:
import pandas as pd

# Load your test.csv file (make sure it's in the same folder as your notebook)
df_test = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/test.csv")

# Combine prompt + responses into one text_input column, matching training format
df_test["text_input"] = (
    "Prompt: " + df_test["prompt"] +
    " \nResponse A: " + df_test["response_a"] +
    " \nResponse B: " + df_test["response_b"]
)

# Keep only 'id' and 'text_input' for prediction input
df_test_formatted = df_test[["id", "text_input"]]

# Save to a new CSV file for model prediction
df_test_formatted.to_csv("test_formatted.csv", index=False)

print("File 'test_formatted.csv' saved successfully!")


File 'test_formatted.csv' saved successfully!


In [ ]:
# vectorizing text

In [7]:
pip install pandas scikit-learn xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/09/c9/5f0be8e51d55df60a1bd7d09e7b05380e04c38de9554105f6cacffac3886/xgboost-3.0.2-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd

df = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/train_formatted.csv")
print(df.columns.tolist())


['text_input', 'label']


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fill NaNs just in case
df["text_input"] = df["text_input"].fillna("")

# Vectorize
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["text_input"])
y = df["label"]


In [23]:
print(df_filtered["text_input"].apply(type).value_counts())


Series([], Name: count, dtype: int64)


In [27]:
df_filtered["text_input"] = df_filtered["text_input"].astype(str)


In [28]:
for i, txt in enumerate(df_filtered["text_input"].head(10)):
    print(f"--- Sample {i} ---")
    print(repr(txt))
    print()


In [40]:
# Map string labels to integers
label_map = {"A": 0, "B": 1}

# Filter out ties and map
df = df[df["label"].isin(label_map.keys())].copy()
df["label"] = df["label"].map(label_map)

print("Filtered rows:", len(df))
print(df["label"].value_counts())


Filtered rows: 39716
label
0    20064
1    19652
Name: count, dtype: int64


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Clean text_input
df["text_input"] = df["text_input"].fillna("").astype(str).str.strip()

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["text_input"])
y = df["label"]


In [46]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred, target_names=["A", "B"]))


/Users/pennatarunkumar/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:03:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.513595166163142
              precision    recall  f1-score   support

           A       0.51      0.58      0.54      1980
           B       0.52      0.45      0.48      1992

    accuracy                           0.51      3972
   macro avg       0.51      0.51      0.51      3972
weighted avg       0.51      0.51      0.51      3972



In [47]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load data
df = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/train_formatted.csv")

print(f"Total rows before filtering: {len(df)}")
print(df.columns.tolist())
print(df.head())

# Step 2: Map string labels to integers and filter out ties
label_map = {"A": 0, "B": 1}
df = df[df["label"].isin(label_map.keys())].copy()
df["label"] = df["label"].map(label_map)

print(f"Total rows after filtering ties: {len(df)}")
print(df["label"].value_counts())

# Step 3: Clean text column
df["text_input"] = df["text_input"].fillna("").astype(str).str.strip()

# Step 4: Vectorize text with TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["text_input"])
y = df["label"]

print(f"TF-IDF matrix shape: {X.shape}")

# Step 5: Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# Step 6: Initialize and train XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Step 7: Predict and evaluate
y_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred, target_names=["A", "B"]))


Total rows before filtering: 57477
['text_input', 'label']
                                          text_input       label
0  Prompt: ["Is it morally right to try to have a...           A
1  Prompt: ["What is the difference between marri...           B
2  Prompt: ["explain function calling. how would ...  winner_tie
3  Prompt: ["How can I create a test set for a ve...           A
4  Prompt: ["What is the best way to travel from ...           B
Total rows after filtering ties: 39716
label
0    20064
1    19652
Name: count, dtype: int64
TF-IDF matrix shape: (39716, 5000)


/Users/pennatarunkumar/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:04:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.513595166163142

Classification Report:
               precision    recall  f1-score   support

           A       0.51      0.58      0.54      1980
           B       0.52      0.45      0.48      1992

    accuracy                           0.51      3972
   macro avg       0.51      0.51      0.51      3972
weighted avg       0.51      0.51      0.51      3972



In [ ]:
Test data & Predict

In [49]:
# Load test data
test_df = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/test_formatted.csv")

print(f"Test rows: {len(test_df)}")
print(test_df.columns.tolist())
print(test_df.head())

# If test data has separate prompt and responses, create 'text_input' same way as train
# For example, if columns are 'prompt', 'response_a', 'response_b', do:

# test_df["text_input"] = (
#     "Prompt: " + test_df["prompt"].astype(str) +
#     "\nResponse A: " + test_df["response_a"].astype(str) +
#     "\nResponse B: " + test_df["response_b"].astype(str)
# )

# If test already has 'text_input' column like train, then just:
test_df["text_input"] = test_df["text_input"].fillna("").astype(str).str.strip()

# Vectorize test text using the same TF-IDF vectorizer fitted on train data
X_test = tfidf.transform(test_df["text_input"])

# Predict with trained model
test_preds = model.predict(X_test)

# Map predictions back to labels "A" or "B"
inv_label_map = {0: "A", 1: "B"}
test_df["winner"] = [inv_label_map[p] for p in test_preds]

print(test_df[["winner"]].head())


Test rows: 3
['id', 'text_input']
        id                                         text_input
0   136060  Prompt: ["I have three oranges today, I ate an...
1   211333  Prompt: ["You are a mediator in a heated polit...
2  1233961  Prompt: ["How to initialize the classification...
  winner
0      A
1      B
2      B


In [50]:
sample_sub = pd.read_csv("/Users/pennatarunkumar/Documents/llm-classification-finetuning/sample_submission.csv")


submission = sample_sub.copy()
submission["winner"] = test_df["winner"]

submission.to_csv("final_submission.csv", index=False)
print("Saved final_submission.csv")


Saved final_submission.csv
